# Single Fully-Connected Network (TensorFlow)

<a href="https://colab.research.google.com/github/VertaAI/modeldb/blob/master/client/workflows/examples/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

This example features:
- **Tensorflow**'s Keras API for building neural networks
- **verta**'s Python client logging observations and artifacts during validation
- **verta**'s Python client retrieving the loss/accuracy plot and the trained model from the validation process

In [2]:
HOST = "app.verta.ai"

PROJECT_NAME = "MNIST Multiclassification"
EXPERIMENT_NAME = "FC-NN"

In [3]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 

## Imports

In [4]:
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import time

import six

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets

import tensorflow as tf
from tensorflow import keras

---

# Log Workflow

## Prepare Data

In [5]:
data = datasets.load_digits()

X = data['data']
y = data['target']

In [6]:
df = pd.DataFrame(np.hstack((X, y.reshape(-1, 1))),
                  columns=["pixel_{}".format(i) for i in range(X.shape[-1])] + ['digit'])

df.head()

## Prepare Hyperparameters

In [7]:
hyperparams = {
    'hidden_size': 512,
    'dropout': 0.2,
    'batch_size': 1024,
    'num_epochs': 10,
    'optimizer': "adam",
    'loss': "sparse_categorical_crossentropy",
    'validation_split': 0.1,
}

## Instantiate Client

In [8]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

## Begin Experiment Run

In [9]:
run = client.set_experiment_run()

## Define and Log Model

In [10]:
run.log_hyperparameters(hyperparams)

model = keras.models.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(hyperparams['hidden_size'], activation=tf.nn.relu))
model.add(keras.layers.Dropout(rate=hyperparams['dropout']))
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer=hyperparams['optimizer'],
              loss=hyperparams['loss'],
              metrics=['accuracy'])

## Log Data

In [11]:
run.log_dataset("train_data", df)

## Run and Log Training

In [12]:
def log_validation_callback(epoch, logs):  # Keras will call this each epoch
    run.log_observation("train_loss", float(logs['loss']))
    run.log_observation("train_acc", float(logs['acc']))
    run.log_observation("val_loss", float(logs['val_loss']))
    run.log_observation("val_acc", float(logs['val_acc']))

model.fit(X, y,
          validation_split=hyperparams['validation_split'],
          batch_size=hyperparams['batch_size'], epochs=hyperparams['num_epochs'],
          callbacks=[keras.callbacks.LambdaCallback(on_epoch_end=log_validation_callback)])

## Create and Store Plot of Training Observations

In [13]:
plt.plot([obs[0] for obs in run.get_observation("val_acc")], label="val")
plt.plot([obs[0] for obs in run.get_observation("train_acc")], label="train")
plt.ylim(0, 1)
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.legend(loc='best')
run.log_image("validation_plot", plt)

plt.show()

## Log Model

In [14]:
run.log_model(model, model_api=ModelAPI(X, model.predict(X)))

---

# Revisit Workflow

## Retrieve Image

In [15]:
run.get_image("validation_plot")

## Retrieve Model

In [16]:
model = run.get_model()

## Calculate and Log Accuracy on Full Training Set

In [17]:
train_loss, train_acc = model.evaluate(X, y)
run.log_metric("train_loss", train_loss)
run.log_metric("train_acc", train_acc)

## Log Requirements for Deployment

In [18]:
run.log_requirements(["tensorflow"])

---

# Make Live Predictions

## Deploy Model Through Web App

In [19]:
run

## Load Deployed Model

In [20]:
from verta._demo_utils import DeployedModel

deployed_model = DeployedModel(HOST, run.id)

## Query Deployed Model

In [21]:
for x in itertools.cycle(X.tolist()):
    print(deployed_model.predict([x]))
    time.sleep(.5)

---